# Libraries initialization

In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd

from kaggle_datasets import KaggleDatasets

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
print(tf.__version__)
%matplotlib inline

# Coopied data loading funcionality

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('flower-classification-with-tpus')
IMAGE_SIZE = [192, 192]

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192/'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + 'train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + 'val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + 'test/*.tfrec')

In [ ]:
GCS_PATH

In [ ]:
TRAINING_FILENAMES

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 150

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int64)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_TESTING_IMAGES = count_data_items(TEST_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)

In [ ]:
print(f"No of training images: {NUM_TRAINING_IMAGES}")
print(f"No of validation images: {NUM_VALIDATION_IMAGES}")
print(f"No of test images: {NUM_TESTING_IMAGES}")

# Example data set images

In [ ]:
plt.figure(figsize=(10,10))
prefetch_ds = get_validation_dataset().take(25)
iterator = prefetch_ds.enumerate().as_numpy_iterator()

for i, data in iterator:
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data[0][0], cmap=plt.cm.binary)
    plt.xlabel(CLASSES[data[1][0]])
plt.show()


# Models

In [ ]:
MODELS_MAP = {}
MODELS_HISTORY = {}

# Base model

In [ ]:
basic_model = models.Sequential()
basic_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 192, 3)))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Conv2D(16, (3, 3), activation='relu'))
basic_model.add(layers.MaxPooling2D((2, 2)))
basic_model.add(layers.Conv2D(8, (3, 3), activation='relu'))
basic_model.add(layers.Flatten())
basic_model.add(layers.Dense(512, activation='relu'))
basic_model.add(layers.Dense(len(CLASSES), activation='relu'))

basic_model.summary()

MODELS_MAP["basic"] = basic_model

# Base with dropout

In [ ]:
dropout_model = models.Sequential()
dropout_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 192, 3)))
dropout_model.add(layers.MaxPooling2D((2, 2)))
dropout_model.add(layers.Conv2D(16, (3, 3), activation='relu'))
dropout_model.add(layers.MaxPooling2D((2, 2)))
dropout_model.add(layers.Conv2D(8, (3, 3), activation='relu'))
dropout_model.add(layers.Dropout(0.2))
dropout_model.add(layers.Flatten())
dropout_model.add(layers.Dense(512, activation='relu'))
dropout_model.add(layers.Dense(len(CLASSES), activation='relu'))

dropout_model.summary()

MODELS_MAP["dropout"] = dropout_model

# Base dropout with different filters

In [ ]:
filtered_model = models.Sequential()
filtered_model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(192, 192, 3)))
filtered_model.add(layers.MaxPooling2D((2, 2)))
filtered_model.add(layers.Conv2D(16, (3, 3), activation='relu'))
filtered_model.add(layers.MaxPooling2D((2, 2)))
filtered_model.add(layers.Conv2D(8, (1, 1), activation='relu'))
filtered_model.add(layers.Dropout(0.2))
filtered_model.add(layers.Flatten())
filtered_model.add(layers.Dense(512, activation='relu'))
filtered_model.add(layers.Dense(len(CLASSES), activation='relu'))

filtered_model.summary()

MODELS_MAP["filtered"] = filtered_model

# VGG Like

In [ ]:
vgg_like = models.Sequential()
vgg_like.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(192, 192, 3)))
vgg_like.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like.add(layers.GlobalAveragePooling2D())
vgg_like.add(layers.Dense(512, activation='relu'))
vgg_like.add(layers.Dense(512, activation='relu'))
vgg_like.add(layers.Dense(len(CLASSES), activation='softmax'))


vgg_like.summary()

MODELS_MAP["vgg-like"] = vgg_like

# Base With 1x1

In [ ]:
x1_model = models.Sequential()
x1_model.add(layers.Conv2D(32, (1, 1), activation='relu', input_shape=(192, 192, 3)))
x1_model.add(layers.Conv2D(32, (7, 7), activation='relu'))
x1_model.add(layers.MaxPooling2D((2, 2)))
x1_model.add(layers.Conv2D(32, (1, 1), activation='relu'))
x1_model.add(layers.Conv2D(32, (5, 5), activation='relu'))
x1_model.add(layers.MaxPooling2D((2, 2)))
x1_model.add(layers.Conv2D(32, (1, 1), activation='relu'))
x1_model.add(layers.Conv2D(32, (3, 3), activation='relu'))
x1_model.add(layers.Dropout(0.2))
x1_model.add(layers.Flatten())
x1_model.add(layers.Dense(512, activation='relu'))
x1_model.add(layers.Dense(len(CLASSES), activation='relu'))

x1_model.summary()

MODELS_MAP["1x1"] = x1_model

# VGG-Like with 1x1

In [ ]:
vgg_like_nxn = models.Sequential()
vgg_like_nxn.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(192, 192, 3)))
vgg_like_nxn.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn.add(layers.Conv2D(16, (1,1), padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn.add(layers.Conv2D(32, (1,1), padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn.add(layers.Conv2D(64, (1,1), padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn.add(layers.Conv2D(64, (1,1), padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn.add(layers.GlobalAveragePooling2D())
vgg_like_nxn.add(layers.Dense(512, activation='relu'))
vgg_like_nxn.add(layers.Dense(512, activation='relu'))
vgg_like_nxn.add(layers.Dense(len(CLASSES), activation='softmax'))


vgg_like_nxn.summary()

MODELS_MAP["vgg-like-1x1"] = vgg_like_nxn

# VGG Like 1x1 with nxn

In [ ]:
vgg_like_nxn_1x1 = models.Sequential()
vgg_like_nxn_1x1.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(192, 192, 3)))
vgg_like_nxn_1x1.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn_1x1.add(layers.Conv2D(16, (1,1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(16, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(16, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn_1x1.add(layers.Conv2D(32, (1,1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(32, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(32, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(32, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(32, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn_1x1.add(layers.Conv2D(64, (1,1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn_1x1.add(layers.Conv2D(64, (1,1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (3, 1), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.Conv2D(64, (1, 3), padding='same', activation='relu'))
vgg_like_nxn_1x1.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_nxn_1x1.add(layers.GlobalAveragePooling2D())
vgg_like_nxn_1x1.add(layers.Dense(512, activation='relu'))
vgg_like_nxn_1x1.add(layers.Dense(512, activation='relu'))
vgg_like_nxn_1x1.add(layers.Dense(len(CLASSES), activation='softmax'))


vgg_like_nxn_1x1.summary()

MODELS_MAP["vgg-like-nxn"] = vgg_like_nxn_1x1

# Pseudo xception

In [ ]:
def conv_bn(x, filters, kernel_size, strides=1):
    
    x = layers.Conv2D(filters=filters, 
               kernel_size = kernel_size, 
               strides=strides, 
               padding = 'same', 
               use_bias = False)(x)
    x = layers.BatchNormalization()(x)
    return x

def sep_bn(x, filters, kernel_size, strides=1):
    
    x = layers.SeparableConv2D(filters=filters, 
                        kernel_size = kernel_size, 
                        strides=strides, 
                        padding = 'same', 
                        use_bias = False)(x)
    x = layers.BatchNormalization()(x)
    return x

def entry_flow(x):
    
    x = conv_bn(x, filters =32, kernel_size =3, strides=2)
    x = layers.ReLU()(x)
    x = conv_bn(x, filters =64, kernel_size =3, strides=1)
    tensor = layers.ReLU()(x)
    
    x = sep_bn(tensor, filters = 128, kernel_size =3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 128, kernel_size =3)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=128, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=256, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=728, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    return x

def middle_flow(tensor, nums):
    
    for _ in range(nums):
        x = layers.ReLU()(tensor)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = layers.ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = layers.ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = layers.ReLU()(x)
        tensor = layers.Add()([tensor,x])
        
    return tensor

def exit_flow(tensor):
    
    x = layers.ReLU()(tensor)
    x = sep_bn(x, filters = 728,  kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 1024,  kernel_size=3)
    x = layers.MaxPool2D(pool_size = 3, strides = 2, padding ='same')(x)
    
    tensor = conv_bn(tensor, filters =1024, kernel_size=1, strides =2)
    x = layers.Add()([tensor,x])
    
    x = sep_bn(x, filters = 1536,  kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 2048,  kernel_size=3)
    x = layers.GlobalAvgPool2D()(x)
    
    x = layers.Dense(units = 1000, activation = 'softmax')(x)
    
    return x

input = layers.Input(shape = (192,192,3))
x = entry_flow(input)
x = middle_flow(x, 4)
output = exit_flow(x)

model_xce = models.Model(inputs=input, outputs=output)
model_xce.summary()

MODELS_MAP["xce"] = model_xce

# Base with nx1 1xn

In [ ]:
xn_model = models.Sequential()
xn_model.add(layers.Conv2D(32, (7, 1), activation='relu', input_shape=(192, 192, 3)))
xn_model.add(layers.Conv2D(32, (1, 7), activation='relu'))
xn_model.add(layers.MaxPooling2D((2, 2)))
xn_model.add(layers.Conv2D(32, (5, 1), activation='relu'))
xn_model.add(layers.Conv2D(32, (1, 5), activation='relu'))
xn_model.add(layers.MaxPooling2D((2, 2)))
xn_model.add(layers.Conv2D(32, (3, 1), activation='relu'))
xn_model.add(layers.Conv2D(32, (1, 3), activation='relu'))
xn_model.add(layers.Dropout(0.2))
xn_model.add(layers.Flatten())
xn_model.add(layers.Dense(512, activation='relu'))
xn_model.add(layers.Dense(len(CLASSES), activation='relu'))

xn_model.summary()

MODELS_MAP["nx1"] = xn_model

# VGG Like with dropout and normalization

In [ ]:
vgg_like_drop = models.Sequential()
vgg_like_drop.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(192, 192, 3)))
vgg_like_drop.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.BatchNormalization())
vgg_like_drop.add(layers.Dropout(0.1))
vgg_like_drop.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_drop.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.BatchNormalization())
vgg_like_drop.add(layers.Dropout(0.1))
vgg_like_drop.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_drop.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.BatchNormalization())
vgg_like_drop.add(layers.Dropout(0.1))
vgg_like_drop.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.BatchNormalization())
vgg_like_drop.add(layers.Dropout(0.1))
vgg_like_drop.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_drop.add(layers.BatchNormalization())
vgg_like_drop.add(layers.Dropout(0.1))
vgg_like_drop.add(layers.MaxPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_drop.add(layers.GlobalAveragePooling2D())
vgg_like_drop.add(layers.Dense(512, activation='relu'))
vgg_like_drop.add(layers.Dense(512, activation='relu'))
vgg_like_drop.add(layers.Dense(len(CLASSES), activation='softmax'))


vgg_like_drop.summary()

MODELS_MAP["vgg-like-drop"] = vgg_like_drop

# Previous VGG with avg pooling

In [ ]:
vgg_like_avg = models.Sequential()
vgg_like_avg.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu', input_shape=(192, 192, 3)))
vgg_like_avg.add(layers.Conv2D(filters=8, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.BatchNormalization())
vgg_like_avg.add(layers.Dropout(0.1))
vgg_like_avg.add(layers.AvgPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_avg.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.BatchNormalization())
vgg_like_avg.add(layers.Dropout(0.1))
vgg_like_avg.add(layers.AvgPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_avg.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.BatchNormalization())
vgg_like_avg.add(layers.Dropout(0.1))
vgg_like_avg.add(layers.AvgPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.BatchNormalization())
vgg_like_avg.add(layers.Dropout(0.1))
vgg_like_avg.add(layers.AvgPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
vgg_like_avg.add(layers.BatchNormalization())
vgg_like_avg.add(layers.Dropout(0.1))
vgg_like_avg.add(layers.AvgPool2D(pool_size=2, strides=2, padding ='same'))

vgg_like_avg.add(layers.GlobalAveragePooling2D())
vgg_like_avg.add(layers.Dense(512, activation='relu'))
vgg_like_avg.add(layers.Dense(512, activation='relu'))
vgg_like_avg.add(layers.Dense(len(CLASSES), activation='softmax'))


vgg_like_avg.summary()

MODELS_MAP["vgg-like-avg"] = vgg_like_avg

# Xception with different filters

In [ ]:
def conv_bn(x, filters, kernel_size, strides=1):
    x = layers.Conv2D(filters=filters, 
               kernel_size = kernel_size, 
               strides=strides, 
               padding = 'same', 
               use_bias = False)(x)
    x = layers.BatchNormalization()(x)
    return x

def sep_bn(x, filters, kernel_size, strides=1):
    x = layers.SeparableConv2D(filters=filters, 
                        kernel_size = kernel_size, 
                        strides=strides, 
                        padding = 'same', 
                        use_bias = False)(x)
    x = layers.BatchNormalization()(x)
    return x

def entry_flow(x):
    
    x = conv_bn(x, filters =32, kernel_size =7, strides=2)
    x = layers.ReLU()(x)
    x = conv_bn(x, filters =64, kernel_size =7, strides=1)
    tensor = layers.ReLU()(x)
    
    x = sep_bn(tensor, filters = 128, kernel_size =7)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 128, kernel_size =7)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=128, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=7)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=7)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=256, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=7)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=7)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=728, kernel_size = 1,strides=2)
    x = layers.Add()([tensor,x])
    return x

def middle_flow(tensor, nums):
    
    for _ in range(nums):
        x = layers.ReLU()(tensor)
        x = sep_bn(x, filters = 728, kernel_size = 5)
        x = layers.ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 5)
        x = layers.ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 5)
        x = layers.ReLU()(x)
        tensor = layers.Add()([tensor,x])
        
    return tensor

def exit_flow(tensor):
    
    x = layers.ReLU()(tensor)
    x = sep_bn(x, filters = 728,  kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 1024,  kernel_size=3)
    x = layers.MaxPool2D(pool_size = 3, strides = 2, padding ='same')(x)
    
    tensor = conv_bn(tensor, filters =1024, kernel_size=1, strides =2)
    x = layers.Add()([tensor,x])
    
    x = sep_bn(x, filters = 1536,  kernel_size=3)
    x = layers.ReLU()(x)
    x = sep_bn(x, filters = 2048,  kernel_size=3)
    x = layers.GlobalAvgPool2D()(x)
    
    x = layers.Dense(units = 1000, activation = 'softmax')(x)
    
    return x

input = layers.Input(shape = (192,192,3))
x = entry_flow(input)
x = middle_flow(x, 4)
output = exit_flow(x)

model_xce_nxn = models.Model(inputs=input, outputs=output)
model_xce_nxn.summary()

MODELS_MAP["xce_nxn"] = model_xce_nxn

# Models training

In [ ]:
nns = MODELS_MAP.keys()
#nns = ["xce_nxn"]

for name in nns:
    print(f"training: {name}")
    MODELS_MAP[name].compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    MODELS_HISTORY[name] = MODELS_MAP[name].fit(get_training_dataset(), epochs=25, 
                        validation_data=get_validation_dataset(), steps_per_epoch=BATCH_SIZE)

In [ ]:
def generate_model_summary(key):
    history, model = MODELS_HISTORY[key], MODELS_MAP[key]
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')


    return model.evaluate(get_validation_dataset(), verbose=2)

# Results

## Basic Model

In [ ]:
generate_model_summary("basic")

## Basic with dropout

In [ ]:
generate_model_summary("dropout")

## Basic filtered

In [ ]:
generate_model_summary("filtered")

## VGG Like

In [ ]:
generate_model_summary("vgg-like")

## 1x1 Convs

In [ ]:
generate_model_summary("1x1")

## xception

In [ ]:
generate_model_summary("xce")

## nx1 1xn

In [ ]:
generate_model_summary("nx1")

## VGG Like 1x1

In [ ]:
generate_model_summary("vgg-like-1x1")

## VGG like with 1x1 nxn

In [ ]:
generate_model_summary("vgg-like-nxn")

## VGG like with drop and normalization

In [ ]:
generate_model_summary("vgg-like-drop")

# VGG like with avg

In [ ]:
generate_model_summary("vgg-like-avg")

## Exception with different filters

In [ ]:
generate_model_summary("xce_nxn")